In [2]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

mercari_df = pd.read_csv(r'C:\Users\HANA\PycharmProjects\HANATOUR\NLP\TEXT_Example\mercari_train.tsv', sep='\t')
# print(mercari_df.shape)
# print(mercari_df.head(3))
print(mercari_df.info())

import matplotlib.pyplot as plt
import seaborn as sns

y_train_df = mercari_df['price']
# plt.figure(figsize=(6,4))
# sns.distplot(y_train_df, kde=False)
# plt.show()

import numpy as np

y_train_df = np.log1p(y_train_df)
# sns.distplot(y_train_df, kde=False)
# plt.show()
mercari_df['price'] = np.log1p(mercari_df['price'])
# print(mercari_df['price'].head(3))

# print(mercari_df['shipping'].value_counts())
# print(mercari_df['item_condition_id'].value_counts())
# boolean_cond = mercari_df['item_description']=='No description yet'
# print(mercari_df[boolean_cond]['item_description'].count())

def split_cat(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Other_Null','Other_Null', 'Other_Null']

mercari_df['cat_dae'], mercari_df['cat_jung'], mercari_df['cat_so'] = \
    zip(*mercari_df['category_name'].apply(lambda x : split_cat(x)))
#
# print(mercari_df['cat_dae'].value_counts())
# print(mercari_df['cat_jung'].value_counts())
# print(mercari_df['cat_so'].value_counts())
#
# print(mercari_df['cat_jung'].nunique())
# print(mercari_df['cat_so'].nunique())

mercari_df['brand_name']= mercari_df['brand_name'].fillna(value='Other_Null')
mercari_df['category_name']= mercari_df['category_name'].fillna(value='Other_Null')
mercari_df['item_description']= mercari_df['item_description'].fillna(value='item_description')

# print(mercari_df.isnull().sum())

# print(mercari_df['brand_name'].nunique())
# print(mercari_df['brand_name'].value_counts()[:5])

# print(mercari_df['name'].nunique())
# print(mercari_df['name'].value_counts()[:10])

# print(mercari_df['item_description'].str.len().mean())
# print(mercari_df['item_description'][:3])

cnt_vec = CountVectorizer()
X_name = cnt_vec.fit_transform(mercari_df.name)

tfidf_descp = TfidfVectorizer(max_features=5000, ngram_range=(1,3), stop_words='english')
X_descp = tfidf_descp.fit_transform(mercari_df['item_description'])

print(X_name.shape)
print(X_descp.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   category_name      1476208 non-null  object 
 4   brand_name         849853 non-null   object 
 5   price              1482535 non-null  float64
 6   shipping           1482535 non-null  int64  
 7   item_description   1482531 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB
None
(1482535, 105757)
(1482535, 5000)


In [4]:
from sklearn.preprocessing import LabelBinarizer

#각 피처를 희소 행렬 원-핫 인코딩 변환
lb_brand_name = LabelBinarizer(sparse_output=True)
X_brand = lb_brand_name.fit_transform(mercari_df['brand_name'])

lb_item_cond_id = LabelBinarizer(sparse_output=True)
X_item_cond_id = lb_item_cond_id.fit_transform(mercari_df['item_condition_id'])

lb_shipping = LabelBinarizer(sparse_output=True)
X_shipping = lb_shipping.fit_transform(mercari_df['shipping'])

lb_cat_dae = LabelBinarizer(sparse_output=True)
X_cat_dae = lb_cat_dae.fit_transform(mercari_df['cat_dae'])

lb_cat_jung = LabelBinarizer(sparse_output=True)
X_cat_jung = lb_cat_jung.fit_transform(mercari_df['cat_jung'])

lb_cat_so = LabelBinarizer(sparse_output=True)
X_cat_so = lb_cat_so.fit_transform(mercari_df['cat_so'])

In [8]:
print(type(X_brand), type(X_item_cond_id))
print(X_brand.shape, X_item_cond_id.shape)

<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'>
(1482535, 4810) (1482535, 5)


In [5]:
from scipy.sparse import hstack
import gc

sparse_matrix_list = (X_name, X_descp, X_brand, X_item_cond_id, X_shipping, X_cat_dae, X_cat_jung, X_cat_so)

#hstack 함수를 이용해 인코딩과 벡터화를 수행한데이터 세트를 모두 결합
X_features_sparse = hstack(sparse_matrix_list).tocsr()
print(type(X_features_sparse), X_features_sparse)

#데이터 세트가 메모리를 많이 차지하므로 사용 목적이 끝났으면 바로 메모리에서 삭제
del X_features_sparse
gc.collect()

<class 'scipy.sparse.csr.csr_matrix'>   (0, 23722)	1.0
  (0, 64986)	1.0
  (0, 79003)	1.0
  (0, 85748)	1.0
  (0, 86779)	1.0
  (0, 103761)	1.0
  (0, 106993)	1.0
  (0, 113955)	1.0
  (0, 115569)	1.0
  (0, 115572)	1.0
  (0, 115578)	1.0
  (0, 115687)	1.0
  (0, 116461)	1.0
  (1, 16338)	1.0
  (1, 23522)	1.0
  (1, 52853)	1.0
  (1, 78554)	1.0
  (1, 106042)	0.2923025986506634
  (1, 106326)	0.1545247839797804
  (1, 106485)	0.2693740427346944
  (1, 106752)	0.11408146464636713
  (1, 106797)	0.34052344662053446
  (1, 107599)	0.13631239778791435
  (1, 107601)	0.16904154020714
  (1, 108007)	0.34405600915534873
  :	:
  (1482534, 18128)	1.0
  (1482534, 28824)	1.0
  (1482534, 59621)	1.0
  (1482534, 68340)	1.0
  (1482534, 100265)	1.0
  (1482534, 101248)	1.0
  (1482534, 107316)	0.18887043857386185
  (1482534, 107327)	0.3039205565829163
  (1482534, 107328)	0.3994260059181747
  (1482534, 107421)	0.1428357765317564
  (1482534, 107447)	0.17685940581315582
  (1482534, 108568)	0.10720372690142944
  (1482534, 1086

68

In [11]:
def rmsle(y, y_pred):
    #underflow, overflow를 막기 위해 log가 아닌 log1p로 rmsle계산
    return np.sqrt(np.power(np.log1p(y)-np.log1p(y_pred), 2))

def evaluate_org_price(y_test, preds):
    #원본 데이터는 log1p로 변환되었으므로 exmpm1로 원복 필요
    preds_exmpm = np.expm1(preds)
    y_test_exmpm = np.expm1((y_test))

    #rmsle로 RMSLE값 추출
    rmsle_result = rmsle(y_test_exmpm, preds_exmpm)
    return rmsle_result

import gc
from scipy.sparse import hstack

def model_train_predict(model, matrix_ist):
    #scipy.sparse 모듈의 hstack을 이용해 희소 행렬 결합
    X = hstack(matrix_ist).tocsr()

    X_train, X_test, y_train, y_test = train_test_split(X, mercari_df['price'], test_size=0.2, random_state=156)

    #모델 학습 및 예측
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    del X, X_train, X_test, y_train
    gc.collect()

    return preds, y_test

linear_model = Ridge(solver='lsqr', fit_intercept=False)
sparse_matrix_list = (X_name, X_brand, X_item_cond_id, X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds, y_test = model_train_predict(model=linear_model, matrix_ist=sparse_matrix_list)
print(evaluate_org_price(y_test, linear_preds))

sparse_matrix_list = (X_descp, X_name, X_brand, X_item_cond_id, X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds, y_test = model_train_predict(model=linear_model, matrix_ist=sparse_matrix_list)
print(evaluate_org_price(y_test, linear_preds))


1153590    0.154335
415884     0.226514
863982     0.377205
766390     0.614836
1300185    0.196881
             ...   
1391089    0.382670
1209308    0.197716
508599     0.377310
825361     0.393537
77         0.066022
Name: price, Length: 296507, dtype: float64
1153590    0.473438
415884     0.118772
863982     0.414775
766390     0.434339
1300185    0.114066
             ...   
1391089    0.153560
1209308    0.088183
508599     0.168213
825361     0.379689
77         0.248231
Name: price, Length: 296507, dtype: float64
